<a href="https://colab.research.google.com/github/govind17/Generative-Models-TensorFlow/blob/main/Assignment_1/LGM_Assignement_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Team


1.   Govind Shukla (235192)
2.   Meghana Rao (234907)
3.   Behnam Ensan(230213)

# Testing Illness
Consider a dangerous and/or common illness that people are being tested for to recognize it early (e.g. cancer) and/or prevent its spread (e.g. COVID). The test is either positive or negative. We make the following assumptions:

* About 1% of the population has the illness. That is, any given person has a 1% “a priori” probability of being sick.
* If a sick person is tested, the test returns a positive result 99.9% of the time.
* If a healthy person is tested, the test still returns a positive result 1% of the time.

You take part in a study where a random, representative sample of the population is tested for the illness. Your test result is positive. What is the probability that you have the illness?

In [11]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB

In [12]:
def generateData():
  health_status = []
  healthy = 'Healthy'
  sick = 'Sick'
  negative = 'Negative'
  positive = 'Positive'
  health_status.extend([healthy]*1000)
  health_status.extend([sick]*1000)
  health_test= []
  health_test.extend([negative]*990)
  health_test.extend([positive]*10)
  health_test.extend([positive]*990)
  health_test.extend([negative]*10)
  return pd.DataFrame(list(zip(health_status, health_test)),
               columns =['Health-status', 'Illness-Test'])
  
data_df = generateData()

In [13]:
data_df['weights'] = data_df['Health-status'].map({'Healthy' : 99, 'Sick' : 1})
random_sample_1 = data_df.sample(frac=0.3, weights='weights', random_state=42)
sick = random_sample_1.groupby('Health-status').size().div(len(random_sample_1))
print(f'Chance of turning out sick: {sick.iloc[1]}')

Chance of turning out sick: 0.013333333333333334


In [14]:
random_sample_2 = data_df.sample(frac=0.4, weights='weights', random_state=42)
sick = random_sample_2.groupby('Health-status').size().div(len(random_sample_2))
print(f'Chance of turning out sick : {sick.iloc[1]}')

Chance of turning out sick : 0.01875


In [28]:
prior = random_sample_1.groupby('Illness-Test').size().div(len(random_sample_1))
prior

Illness-Test
Negative    0.981667
Positive    0.018333
dtype: float64

In [16]:
likelihood = {}
likelihood['Health-status'] = random_sample_1.groupby(['Illness-Test', 'Health-status']).size().div(len(random_sample_1)).div(prior)
print (likelihood)

{'Health-status': Illness-Test  Health-status
Negative      Healthy          1.000000
Positive      Healthy          0.272727
              Sick             0.727273
dtype: float64}


In [17]:
random_sample_1.drop(['weights'],inplace=True, axis=1)
le = LabelEncoder()
encoded_data = random_sample_1.apply(le.fit_transform)

In [18]:
clf = GaussianNB()
clf.fit(encoded_data.drop(['Illness-Test'], axis=1).values, encoded_data['Illness-Test'].values)

GaussianNB()

In [19]:
X_healthy = np.array([0])
print (clf._joint_log_likelihood(X_healthy.reshape(1,-1)))
print ("Test Prediction of Healthy: ", le.inverse_transform(clf.predict(X_healthy.reshape(1,-1))))

[[11.58964648 -5.44243789]]
Test Prediction of Healthy:  ['Negative']


In [20]:
X_sick = np.array([1])
print (clf._joint_log_likelihood(X_sick.reshape(1,-1)))
print ("Test Prediction of Sick: ", le.inverse_transform(clf.predict(X_sick.reshape(1,-1))))

[[-3.80067567e+10 -4.29660456e+00]]
Test Prediction of Sick:  ['Positive']


# References



1.   https://ovgu-ailab.github.io/lgm2022/ass1.html
2.   https://www.geeksforgeeks.org/naive-bayes-classifiers/
3.   http://deebuls.github.io/Naive-Bayes-Pandas.html